In [20]:
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import time, requests

In [21]:
print ('\033[1m' + 'Scraping active jobs (citywise) from indeed.com\n' + '\033[0m')
print ("Input one by one names of all the desired US cities to find all active jobs posted in that city.\nInput 'done' if all the desired cities are enetered.\n")
time.sleep(0.2)
flag = True
cities=[]
count=1
while (flag):
    temp = str(input('\t'+str(count)+". ").lower())
    temp = temp.replace(' ','+')
    if temp!='done':
        cities.append(temp)
    else:
        flag=False
    count+=1

req_cols = ["Job Title","Company Name","Sponsored","Posted","Salary"]
maxiter = 1000

for city in cities:
    out_df = pd.DataFrame(columns = req_cols)
    for start_valve in range(0,maxiter,10):
        url = "http://www.indeed.com/jobs?l=%s&start=%s"%(city,start_valve,)
        page = requests.get(url)
        time.sleep(0.5)
        soup = BeautifulSoup(page.text,"lxml")
        for div in soup.find_all(name="div",attrs={"class":"row"}):
                job_posted = []
                try:
                    for a in div.find_all(name="a",attrs={"data-tn-element":"jobTitle"}):
                        job_posted.append(a["title"])
                except:
                    job_posted.append("Not Available")
                try:
                    company_name = div.find_all(name="span",attrs={"class":"company"})
                    if len(company_name) > 0:
                        for b in company_name:
                            job_posted.append(b.text.strip())
                    else:
                        alt = div.find_all(name="span",attrs={"class":"result-link-source"})
                        for span in alt:
                            job_posted.append(span.text)
                except:
                    job_posted.append("Not Available")
                try:
                    job_posted.append(div.find(name="span",attrs={"class":" sponsoredGray "}).text)
                except:
                    job_posted.append("Not Sponsored") 
                try:
                    job_posted.append(div.find(name="span",attrs={"class":"date"}).text)
                except:
                    job_posted.append("Not Available")            
                try:
                    job_posted.append(div.find(name="span",attrs={"class":"no-wrap"}).text.strip('\n '))
                except:
                    job_posted.append("Not Available")
                num = (len(out_df) + 1)
                out_df.loc[num] = job_posted
    if out_df.shape[0] !=0:
        out_df.drop_duplicates(subset=None, inplace=True)
        filename = city.replace('+',' ')+" job listings.csv"
        out_df.to_csv(filename,index=False)
print ("\nTask completed. Citywise '.csv files' are saved at the location where this notebook file resides.")

Scraping active jobs (citywise) from indeed.com

Input one by one names of all the desired US cities to find all active jobs posted in that city.
Input 'done' if all the desired cities are enetered.

	1. chicago
	2. gibberish
	3. los angeles
	4. done

Task completed. Citywise '.csv files' are saved at the location where this notebook file resides.


#### Sample Output

In [26]:
out_df.head(10)

,Job Title,Company Name,Sponsored,Posted,Salary
1,Dog Walker,Wag!,Sponsored,Not Available,$30 an hour
2,Property Assessor,Eco Technology Inc,Sponsored,3 days ago,$500 a week
3,Amazon Delivery Driver,1-800 Courier,Sponsored,Not Available,$160 a day
4,Walt Disney Animation Studios - Creative Roles,Studio Entertainment,Not Sponsored,6 days ago,Not Available
5,Administrative Assistant (Confidential) Studen...,Cerritos College,Not Sponsored,15 days ago,"$5,498 - $6,905 a month"
6,"Assistant, Production",NBCUniversal,Not Sponsored,1 day ago,Not Available
7,"Dog Sitter - Flexible Hours, Work From Home, P...",Rover,Not Sponsored,1 day ago,"$1,000 a month"
8,United States Probation and Pretrial Services ...,United States District Court,Not Sponsored,30+ days ago,"$43,686 - $107,955 a year"
9,Clerk,UCLA Health,Not Sponsored,3 hours ago,Not Available
10,Stocker,Costco Wholesale,Not Sponsored,30+ days ago,Not Available
